

# Amazon Bedrock Guardrails로 Strands Agents 보호하기

## 개요
이 예제에서는 [아마존 베드락 가드레일](https://aws.amazon.com/bedrock/guardrails/) 을 사용하여 기본 아마존 베드락 LLM 모델을 보호하는 첫 번째 스트랜드 에이전트를 만드는 방법을 안내합니다.

Amazon Break Guardrails는 대규모로 생성 AI 애플리케이션을 안전하게 구축할 수 있도록 구성 가능한 안전 장치를 제공합니다. Amazon Break에서 지원되는 FM, 미세 조정된 모델, Amazon Break 외부에서 호스팅되는 모델 등 다양한 기초 모델(FM)에 걸쳐 일관되고 표준적인 접근 방식을 사용하는 Guardrails는 업계 최고의 안전 보호 기능을 제공합니다.

Strands Agents를 사용하면 Amazon Break 모델에 Amazon Break Guardrail을 직접 추가할 수 있습니다. Amazon Break 모델을 사용하지 않는 경우 [가드레일 API 적용](https://docs.aws.amazon.com/bedrock/latest/userguide/guardrails-use-independent-api.html) 을 사용하여 모든 모델을 보호할 수 있습니다. 이 경우 방금 배운 고급 처리 기능을 사용하여 파이프라인을 구축해야 합니다.


## Agent Details
<div style="float: left; margin-right: 20px;">
    
|Feature             |Description                                        |
|--------------------|---------------------------------------------------|
|AWS Services used   |Amazon Bedrock Guardrails                          |
|Custom tools created|get_customer_profile, list_customer_purchases, list_customer_tickets, update_customer_profile|
|Agent Structure     |Single agent architecture                          |

</div>


## Architecture

<div style="text-align:left">
    <img src="images/architecture.png" width="85%" />
</div>

## 주요 기능
* **단일 에이전트 아키텍처**: 이 예제는 내장 도구와 사용자 정의 도구와 상호작용하는 단일 에이전트를 생성합니다
* **사용자 정의 도구**: 자신만의 도구를 만드는 방법을 배웁니다
* **기본 LLM으로서의 Amazon Bedrock 모델**: 기본 LLM 모델로 Amazon Bedrock의 Anthropic Claude 3.7을 사용했습니다
* **기본 보호 장치로서의 Amazon Bedrock Guardrails**: Amazon Bedrock Guardrails를 사용하여 에이전트 애플리케이션을 보호합니다

## 설정 및 전제 조건

### 전제 조건
* Python 3.10+
* AWS 계정
* Amazon Bedrock에서 활성화된 Anthropic Claude 3.7
* Amazon Bedrock Guardrails를 생성할 권한이 있는 IAM 역할

이제 Strands Agent에 필요한 패키지를 설치해보겠습니다

In [ ]:
%pip install -U -r requirements.txt

### 종속성 패키지 가져오기

이제 종속성 패키지를 가져와보겠습니다

In [ ]:
import json
import boto3
import os
from strands import Agent, tool
from strands.models import BedrockModel
from customer_profile_tools import get_customer_profile, list_customer_purchases, list_customer_tickets, update_customer_profile
from customer_profiles import CustomerProfileManager, generate_synthetic_profiles

### 데이터 생성

이 예제를 위해 일부 합성 데이터를 생성해보겠습니다. 실제로는 에이전트가 기존 고객 데이터베이스에 연결되지만, 이 예제에서는 JSON 파일을 사용하겠습니다

In [ ]:
 # 필요한 경우 합성 프로필 생성
profile_manager = CustomerProfileManager()
if not os.path.exists("customer_profiles.json"):
    print("합성 고객 프로필 생성 중")
    profiles = generate_synthetic_profiles(10)
    print(f"{len(profiles)}개의 합성 고객 프로필을 생성했습니다")
else:
    print("기존 고객 프로필 사용")
    profiles = list(profile_manager.profiles.values())
    print(f"{len(profiles)}개의 고객 프로필을 로드했습니다")

# 참조용 샘플 프로필 표시
if profiles:
    sample_profile = profiles[0]
    print(f"\n샘플 프로필: {sample_profile.name}")
    print(f"이메일: {sample_profile.email}")
    print(f"고객 ID: {sample_profile.customer_id}")
    print(f"위치: {sample_profile.state}, {sample_profile.country}")
    print(f"구매 내역: {len(sample_profile.purchase_history)}")
    print(f"지원 티켓: {len(sample_profile.support_tickets)}")

## Amazon Bedrock Guardrail 생성

이제 금융 조언 제공을 방지하고 고객 대면 애플리케이션의 일반적인 문제를 필터링하기 위한 Amazon Bedrock Guardrail을 생성해보겠습니다.

가드레일을 생성하려면 먼저 Amazon Bedrock 클라이언트를 구성해야 합니다:

In [ ]:
# AWS 클라이언트 구성
bedrock_client = boto3.client('bedrock')
bedrock_runtime = boto3.client('bedrock-runtime')

이제 `create_guardrail` 메서드를 사용하여 애플리케이션에서 사용할 `no-investment-advice` 가드레일을 생성할 수 있습니다. 이 예제에서 가드레일은 에이전트가 신탁 조언을 제공하는 것을 차단하는 `Fiduciary Advice`에 대한 주제 거부를 포함합니다. 가드레일에는 부적절한 콘텐츠를 필터링하는 기본 콘텐츠 정책과 특정 사전 정의된 단어를 감지하는 단어 정책 구성도 포함됩니다.

In [ ]:
# Bedrock 가드레일 생성
response = bedrock_client.create_guardrail(
    name='no-investment-advice',
    description='모델이 신탁 조언을 제공하는 것을 방지합니다.',
    topicPolicyConfig={
        'topicsConfig': [
            {
                'name': 'Fiduciary Advice',
                'definition': 'Providing personalized advice or recommendations on managing financial assets, investments, or trusts in a fiduciary capacity or assuming related obligations and liabilities.',
                'examples': [
                    'What stocks should I invest in for my retirement?',
                    'Is it a good idea to put my money in a mutual fund?',
                    'How should I allocate my 401(k) investments?',
                    'What type of trust fund should I set up for my children?',
                    'Should I hire a financial advisor to manage my investments?'
                ],
                'type': 'DENY'
            }
        ]
    },
    contentPolicyConfig={
        'filtersConfig': [
            {
                'type': 'SEXUAL',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'VIOLENCE',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'HATE',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'INSULTS',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'MISCONDUCT',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'PROMPT_ATTACK',
                'inputStrength': 'HIGH',
                'outputStrength': 'NONE'
            }
        ]
    },
    wordPolicyConfig={
        'wordsConfig': [
            {'text': 'fiduciary advice'},
            {'text': 'investment recommendations'},
            {'text': 'stock picks'},
            {'text': 'financial planning guidance'},
            {'text': 'portfolio allocation advice'},
            {'text': 'retirement fund suggestions'},
            {'text': 'wealth management tips'},
            {'text': 'trust fund setup'},
            {'text': 'investment strategy'},
            {'text': 'financial advisor recommendations'}
        ],
        'managedWordListsConfig': [
            {
                'type': 'PROFANITY'
            }
        ]
    },
    blockedInputMessaging='죄송하지만 신탁 조언을 제공할 수 없습니다. 개인정보 보호와 보안을 위해 금융 또는 제한된 세부 정보를 포함하지 않고 입력을 수정한 후 다시 시도해 주세요.',
    blockedOutputsMessaging='죄송하지만 신탁 조언을 제공할 수 없습니다. 개인정보 보호와 보안을 위해 금융 또는 제한된 세부 정보를 포함하지 않고 입력을 수정한 후 다시 시도해 주세요.',
)

# Print the response to get the guardrail ID
print("Guardrail ID:", response.get('guardrailId'))
print("Guardrail ARN:", response.get('guardrailArn'))

# Store the guardrail ID for later use


In [ ]:
guardrail_id = response.get('guardrailId')
guardrail_version = "DRAFT"  # 초기 버전은 항상 1

### 가드레일 직접 테스트

가드레일이 예상대로 작동하는지 확인하기 위해 `test_guardrail` 지원 함수를 생성합니다. 이 함수는 `apply_guardrail` 메서드를 사용하여 입력 텍스트를 보호하고 가드레일이 수행한 모든 작업에 대한 정보를 제공합니다

In [ ]:
# 가드레일에 의해 입력/출력이 차단되는지 확인하는 테스트 함수
def test_guardrail(text, source_type='INPUT'):
      response = bedrock_runtime.apply_guardrail(
          guardrailIdentifier=guardrail_id,
          guardrailVersion=guardrail_version,
          source=source_type,  # 'INPUT' 또는 'OUTPUT'이 될 수 있음
          content=[{"text": {"text": text}}]
      )

      # 새로운 응답 형식은 다른 필드를 사용
      print(f"작업: {response.get('action')}")
      print(f"작업 이유: {response.get('actionReason', 'None')}")

      # 콘텐츠가 차단되었는지 확인
      is_blocked = response.get('action') == 'GUARDRAIL_INTERVENED'
      print(f"콘텐츠 {source_type} 차단됨: {is_blocked}")

      if is_blocked:
          # 트리거된 주제 정책 출력
          assessments = response.get('assessments', [])
          if assessments and 'topicPolicy' in assessments[0]:
              print("차단된 주제:", [topic.get('name') for topic in
  assessments[0]['topicPolicy'].get('topics', [])
                                       if topic.get('action') == 'BLOCKED'])

          # 사용 가능한 경우 수정된 출력 인쇄
          if 'outputs' in response and response['outputs']:
              print("수정된 콘텐츠:", response['outputs'][0].get('text', 'None'))

      return response

# 안전한 입력 테스트
print("안전한 입력 테스트:")
test_guardrail("일반적인 금융 이해력 개념에 대해 알려주세요.")

# 차단되어야 하는 입력 테스트
print("\n차단되어야 하는 입력 테스트:")
test_guardrail("은퇴를 위해 어떤 주식에 투자해야 하나요?")

## Strands Agent와 통합

가드레일이 예상대로 작동하는 것을 확인했으므로 이제 Amazon Bedrock Guardrail을 Strands Agent와 통합해보겠습니다. 이는 `guardrail_id`, `guardrail_version` 및 `guardrail_trace`를 설정하여 Bedrock Model 객체를 통해 수행됩니다. 모델 객체가 생성되면 이를 사용하여 에이전트를 생성할 수 있습니다. 이 에이전트에서 몇 가지 사용자 정의 도구를 사용합니다: `get_customer_profile`, `list_customer_purchases`, `list_customer_tickets`, `update_customer_profile`. 구현을 보려면 `customer_profile_tools.py` 파일을 확인하세요

In [ ]:
guardrail_id, guardrail_version

In [ ]:
# 가드레일 구성으로 Bedrock 모델 생성
bedrock_model = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    guardrail_id=guardrail_id,
    guardrail_version=guardrail_version,
    # 디버깅을 위한 추적 정보 활성화
    guardrail_trace="enabled"
)

# 가드레일로 보호된 모델로 에이전트 생성
agent = Agent(
    system_prompt="당신은 소매 제품에 대한 고객 지원을 제공하는 도움이 되는 어시스턴트입니다.",
    model=bedrock_model,
    tools=[
        get_customer_profile,
        list_customer_purchases,
        list_customer_tickets,
        update_customer_profile
    ]
)

### 가드레일이 있는 Strands Agent 테스트

안전한 입력과 위험한 입력 모두로 에이전트를 테스트해보겠습니다. 이를 위해 에이전트의 응답을 처리하고 `stop_reason`이 가드레일 개입 때문인지 확인합니다.

In [ ]:
# 에이전트를 테스트하고 가드레일 개입을 확인하는 도우미 함수
def test_agent_with_guardrail(prompt):
    print(f"\n사용자: {prompt}")

    # 에이전트 응답 가져오기
    response = agent(prompt)

    # 가드레일 개입 확인
    if hasattr(response, 'stop_reason') and response.stop_reason == "guardrail_intervened":
        print("\n ⚠️ 가드레일이 개입했습니다!")
        #print(f"응답: {response}")
    else:
        return response

In [ ]:
# 안전한 질문으로 테스트
test_agent_with_guardrail(
    "제 고객 ID는 CUST100입니다. 최근 구매 내역이 무엇인가요?"
)

In [ ]:
# 투자 추천이나 혐오, 폭력에 대해 묻는 질문으로 테스트
test_agent_with_guardrail(
    "제 주민등록번호는 123-45-6789입니다. 은퇴 계좌를 보호하는 방법을 이해하는 데 도움을 주실 수 있나요?"
)

## 대화 기록 검사

가드레일이 대화에 어떤 영향을 미쳤는지 대화 기록을 살펴보겠습니다:

In [ ]:
# 대화 기록 출력
print(f"대화 기록: {json.dumps(agent.messages, indent=4)}")

## 챗봇에 Bedrock 가드레일 통합

Amazon Bedrock은 Strands Agents SDK와 직접 통합되는 내장 가드레일 프레임워크를 제공합니다. 가드레일이 트리거되면 Strands Agents SDK는 대화 기록에서 사용자 입력을 자동으로 덮어씁니다. 이는 후속 질문이 동일한 질문으로 인해 차단되지 않도록 하기 위함입니다. 이는 guardrail_redact_input 부울과 덮어쓰기 메시지를 변경하는 guardrail_redact_input_message 문자열로 구성할 수 있습니다. 또한 모델의 출력에 대해서도 동일한 기능이 구축되어 있지만 기본적으로 비활성화되어 있습니다. guardrail_redact_output 부울로 이를 활성화하고 guardrail_redact_output_message 문자열로 덮어쓰기 메시지를 변경할 수 있습니다. 다음은 코드에서 Bedrock 가드레일을 활용하는 방법의 예입니다:

In [ ]:
bedrock_model = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    guardrail_id=guardrail_id,         # Bedrock 가드레일 ID
    guardrail_version=guardrail_version,             # 가드레일 버전
    guardrail_trace="enabled",
    guardrail_redact_output = True,        
    guardrail_redact_input = True  ,
    guardrail_redact_input_message = "가드레일이 개입하여 수정됨"     
)

# 가드레일로 보호된 모델로 에이전트 생성
agent = Agent(
    system_prompt="당신은 소매 제품에 대한 고객 지원을 제공하는 도움이 되는 어시스턴트입니다.",
    model=bedrock_model,
    tools=[
        get_customer_profile,
        list_customer_purchases,
        list_customer_tickets,
        update_customer_profile
    ]
)

In [ ]:
# 고객 지원 에이전트를 위한 안전한 질문 테스트
agent(f"고객 ID {sample_profile.customer_id}의 데이터는 무엇인가요?")

In [ ]:
# 가드레일이 개입하고 사용자 정의 메시지로 입력이 수정되는 사용자 입력으로 테스트
agent("401(k) 투자를 어떻게 배분해야 하나요?")

수정된 입력에 대한 사용자 정의 메시지는 다음과 같이 대화 기록에 저장됩니다:

In [ ]:
print(f"대화 기록: {json.dumps(agent.messages, indent=4)}")

## 정리

다음과 같이 가드레일을 삭제해주세요:

In [ ]:
bedrock_client.delete_guardrail(guardrailIdentifier=guardrail_id)

## 축하합니다!

이 노트에서는 다음과 같은 방법을 시연했습니다:

1. 재정적 조언을 제공하지 않는 아마존 베드락 가드레일 만들기
2. 베드락 런타임 API를 사용하여 가드레일을 직접 테스트합니다
3. 가드레일과 스트랜드 에이전트 통합
4. 다양한 입력으로 에이전트를 테스트하여 가드레일이 작동하는지 확인합니다
5. 챗봇의 암반 가드레일과 스트랜드 에이전트를 통합하세요.
5. 가드레일 삭제

가드레일은 AI 응답을 안전하고 준수하며 사용 사례에 적합하게 유지하는 데 도움이 됩니다.